<a href="https://colab.research.google.com/github/shahzaibkhanniazi-dot/MSAI-VetDisease-CRNN-2026/blob/main/MSAI_VetDisease_CRNN_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Initial Setup & Mounting

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

# Setting the seed so I always get the same accuracy
torch.manual_seed(42)

# Mounting my Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Checking if I am using the GPU for faster training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"I am training on: {device}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
I am training on: cuda


In [12]:
# Setting up how I want to process my images
transform = transforms.Compose([
    # Making all images the same size
    transforms.Resize((128, 128)),
    # Randomly flipping images so the model learns better
    transforms.RandomHorizontalFlip(),
    # Turning the images into numbers (tensors) for the model
    transforms.ToTensor(),
    # Normalizing the colors to keep things balanced
    transforms.Normalize((0.5,), (0.5,))
])

# Pointing to the data folder in my Google Drive
data_dir = '/content/drive/MyDrive/MSAI_Project_2026/data'

# Loading the images from my folders
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Splitting my data: 80% for training and 20% for testing my accuracy
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_data, test_data = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Setting up the data loaders to feed images to the model in groups of 32
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

print(f"I found these disease classes: {full_dataset.classes}")

I found these disease classes: ['Bacterial_dermatosis', 'Fungal_infections', 'Healthy', 'Hypersensitivity_allergic_dermatosis']


Step 1: Training the Model from Scratch

In [13]:
# Setting up the loss and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Starting the training process
epochs = 15
print("Beginning training to reach 75% accuracy...")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zeroing the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Tracking accuracy and loss
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    # Calculating accuracy for this epoch
    epoch_acc = 100 * correct_predictions / total_samples
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_acc:.2f}%')

print("Training is complete!")

Beginning training to reach 75% accuracy...
Epoch [1/15], Loss: 1.3289, Accuracy: 28.90%
Epoch [2/15], Loss: 1.2899, Accuracy: 39.09%
Epoch [3/15], Loss: 1.2186, Accuracy: 39.66%
Epoch [4/15], Loss: 1.2475, Accuracy: 43.63%
Epoch [5/15], Loss: 1.1555, Accuracy: 43.34%
Epoch [6/15], Loss: 1.2292, Accuracy: 43.63%
Epoch [7/15], Loss: 1.2496, Accuracy: 45.89%
Epoch [8/15], Loss: 1.1460, Accuracy: 43.06%
Epoch [9/15], Loss: 1.2000, Accuracy: 44.48%
Epoch [10/15], Loss: 1.1907, Accuracy: 44.19%
Epoch [11/15], Loss: 1.1459, Accuracy: 43.06%
Epoch [12/15], Loss: 1.0930, Accuracy: 44.48%
Epoch [13/15], Loss: 1.1319, Accuracy: 47.88%
Epoch [14/15], Loss: 1.1878, Accuracy: 47.03%
Epoch [15/15], Loss: 1.1947, Accuracy: 46.18%
Training is complete!
